# Import, Set Working Directory

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import time

from scipy import stats
from scipy.stats import randint

from imblearn.over_sampling import SMOTE
from boruta import BorutaPy

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet

from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.tree import export_graphviz
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.linear_model import LassoCV
from sklearn.datasets import make_regression

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve 

%matplotlib inline

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
def main():
    print("Current Working Directory " , os.getcwd())
    if os.path.exists("C:/Users/micha.DESKTOP-8HA2IGV/OneDrive/Programming/Propulsion Project/intelligencia_backup/intelligencia") :
        # Change the current working Directory    
        os.chdir("C:/Users/micha.DESKTOP-8HA2IGV/OneDrive/Programming/Propulsion Project/intelligencia_backup/intelligencia")
        print("New Working Directory " , os.getcwd())
    else:
        print("Can't change the Current Working Directory")    
        print("Current Working Directory " , os.getcwd())
if __name__ == '__main__':
    main()

Current Working Directory  C:\Users\micha.DESKTOP-8HA2IGV\OneDrive\Programming\Propulsion Project\intelligencia_backup\intelligencia
New Working Directory  C:\Users\micha.DESKTOP-8HA2IGV\OneDrive\Programming\Propulsion Project\intelligencia_backup\intelligencia


# Class: Data Preparation (dataprep)
Prepares data for feature selection algorithm

In [4]:
class DataPrep:
    """
    Main function -> bulbasaur(path, threshold, nrows = None, usecols = None)
    - input: directory path of gene expression data
    - output: X_train, y_train, x_test, y_test

    Included functions:
    - Read Data
    - Standard Deviation Filter: Set and filter with threshold
    - Split: Train and test split
    - Smote Up: Upsampling to get balanced dataset
    """
    def __init__(self, seed):
        self.seed = seed
    
  # Read Data
    def read_data(self, path, nrows, usecols):
        data = pd.read_csv(path, nrows=nrows, usecols=usecols)
        data.index = data.iloc[:,0]
        data.drop(columns = "Unnamed: 0", inplace = True)
        data.columns = [(re.sub('\.\d+', '', gene)) for gene in data.columns]
        return data
  
  # Filter with Standard Deviation Threshold
    def X_and_y(self, data, threshold):
        #data.describe()
        X = data.drop(columns = 'label')
        X_sd = X.loc[:, X.std() > threshold]
        y = data[["label"]]
        return X_sd, y
  
  # Train Test Split data
    def split(self, X, y, test_size):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=self.seed)
        return X_train, X_test, y_train, y_test
  
  # Upsample unbalanced data
    def smote_up(self, X_train, y_train):
        #print("Before OverSampling, counts of label '1': {}".format(sum(y_train['label']==1)))
        #print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train['label']==0)))

        sm = SMOTE(random_state=self.seed)
        X_train_smote, y_train_smote = sm.fit_sample(X_train, y_train)

        #print('After OverSampling, the shape of train_X: {}'.format(X_train_smote.shape))
        #print('After OverSampling, the shape of train_y: {} \n'.format(y_train_smote.shape))

        #print("After OverSampling, counts of label '1': {}".format(sum(y_train_smote==1)))
        #print("After OverSampling, counts of label '0': {}".format(sum(y_train_smote==0)))
        column_names = X_train.columns

        # Make dataframe again
        X_train_smote = pd.DataFrame(X_train_smote, columns=column_names)
        y_train_smote = pd.DataFrame(y_train_smote, columns=['label'])

        return X_train_smote, y_train_smote
    
    def bulbasaur(self, path, threshold = 2, nrows = None, usecols = None):
        data = self.read_data(path, nrows, usecols)
        X, y = self.X_and_y(data, threshold)
        X_train, X_test, y_train, y_test = self.split(X, y, 0.3)
        X_train_smote, y_train_smote = self.smote_up(X_train, y_train)
        return X_train_smote, y_train_smote, X_test, y_test
    
dataprep = DataPrep(1888)

# Class: Feature Selection (FS)
Applies a variety of feature selectors

In [ ]:
class FeatureSelection:
    """
    Main function -> call_methods(X_train, y_train, X_test, y_test)
    - input: Dataset splitted to train and test size.
    - output: Dictionaries with selected features and feature importances.

    Included functions:
    - RFE: Recursive Feature Elimination
    - Gradient Boost Classifier
    - Elastic Net
    - Boruta Tree: Boruta with Random Forest Classifier at the end
    - Lasso CV: Lasso with crossvalidation
    """
    def __init__(self, seed):
        self.seed = seed
    
    def rfe(self, X_train, y_train, X_test, y_test, n_features = 300, step = 0.2, kernel = "linear"):
        """
        Recursive Feature Elimination - step < 1 is a percentage. Returns selected features.
        Hyperparameter tuning was done in a different notebook
        """
        # Create RFE
        estimator = SVR(kernel=kernel, C = 0.01, gamma = 1e-07)
        selector = RFE(estimator, n_features_to_select = n_features, step=step)
        selector = selector.fit(X_train.to_numpy(), y_train.to_numpy())
        
        # Print Accuracy
        print('Accuracy of RFE: {:.3f}'.format(selector.score(X_test, y_test)))
        
        # Add features and feature importance to dictionary
        selected_features = X_train.columns[selector.support_].tolist()
        
        feature_importances = [1 for x in range(len(selected_features))]
    
        dictionary = {"Recursive Feature Elimination":[selected_features, feature_importances]}
        return dictionary
  
  
    def gradient_boost_classifier(self, X_train, y_train, X_test, y_test, n_features = 300):
        """
        Gradient Boost Classifier with feature importance selection.
        Hypertuned parameters:
        {'learning_rate': 0.5, 'max_depth': 3, 'min_samples_leaf': 7, 'min_samples_split': 0.1, 'n_estimators': 100, 'random_state': 1888, 'subsample': 0.75}
        """
        # Create Gradient Boost Classifier 
        new = GradientBoostingClassifier(learning_rate=0.5, n_estimators=100, max_depth=3,
                                         min_samples_split=0.1, min_samples_leaf=7,\
                                         subsample=0.75, random_state=self.seed)
        new.fit(X_train,y_train)
        predictors = list(X_train)
        feat_imp = pd.Series(new.feature_importances_, predictors).sort_values(ascending=False)[:n_features]
        
        pred = new.predict(X_test)
        
        # Print accuracy
        print('Accuracy of GBM: {:.3f}'.format(new.score(X_test, y_test)))
    
        # Add features and feature importance to dictionary
        importances = new.feature_importances_
        genes = X_test.columns
        
        selected_features_df = pd.DataFrame(importances, index = genes).sort_values(0, ascending = False).head(n_features)
        selected_features = selected_features_df.index.tolist()
        feature_importances = selected_features_df.iloc[:,0].tolist()
        
        dictionary = {"Gradient Boost Classifier":[selected_features, feature_importances]}
        return dictionary
    
    def elastic_net(self, X_train_smote, y_train_res, X_test, y_test, alpha=0.01, l1_ratio=0.5, n_features=300):
        clf = ElasticNet(random_state=self.seed, alpha=alpha, l1_ratio=l1_ratio)
        clf.fit(X_train_smote, y_train_res)

        clf.pred = clf.predict(X_test)

        print("Accuracy of Elastic Net: {:.3f}".format(clf.score(X_test, y_test)))
        #print("Elastic net mean squared error: {:.3f}".format(mean_squared_error(y_test, clf.pred)))

        ft_imp = pd.DataFrame(clf.coef_, index=X_train_smote.columns)
        ft_sort = ft_imp.sort_values(0, ascending=False)
        imp_coef = pd.concat([ft_sort.head(int(n_features/2)), ft_sort.tail(int(n_features/2))])

        selected_features = imp_coef.index.tolist()
        feature_importances = imp_coef.iloc[:,0].tolist()

        dictionary = {"Elastic Net": [selected_features, feature_importances]}

        return dictionary 
  
    def boruta_tree(self, X_train_smote, y_train_res, X_test, y_test):

        for _ in range(1):

            from sklearn.metrics import f1_score # import again because it works like that :)

            # Random Forests for Boruta
            rf_boruta = RandomForestClassifier(n_jobs=-1, random_state=self.seed)

            # Perform Boruta
            boruta = BorutaPy(rf_boruta, n_estimators='auto', verbose=0,
                          alpha=0.005, max_iter=30, perc=100, random_state=self.seed)
            boruta.fit(X_train_smote.values, y_train_res)

            # Select features and fit Logistic Regression

            cols = X_train_smote.columns[boruta.support_]
            X_train_smote = X_train_smote[cols]
            est_boruta = LogisticRegression(random_state=self.seed)
            est_boruta.fit(X_train_smote, y_train_res)

            scores = cross_val_score(est_boruta, X_train_smote, y_train_res, cv=5)

            print("Accuracy of Boruta: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

        # Random Forest for extracting features

        X_filtered = X_train_smote[cols]

        rf = RandomForestClassifier(n_estimators = 10, criterion = 'gini', random_state = self.seed)
        rf.fit(X_filtered, y_train_res)
        rf_pred = rf.predict(X_test[cols])
        print("Accuracy of Boruta Tree: {:.3f}".format(accuracy_score(y_test, rf_pred)))

        feature_names = X_filtered.columns
        rf_coeff = pd.DataFrame({"feature": feature_names,"coefficient": rf.feature_importances_})
        rf_coeff_top = rf_coeff.sort_values(by="coefficient",ascending=False).head(300).set_index("feature")

        selected_features = rf_coeff_top.index.tolist()
        feature_importances = rf_coeff_top.coefficient.tolist()

        dictionary = {"Boruta Tree": [selected_features, feature_importances]}

        return dictionary
    
    def lasso_cv(self, X_train, y_train, X_test, y_test, n_features = 300):
    # LassoCV
        
        lassoCV = LassoCV(cv=3, random_state=1888).fit(X_train, y_train)
        print("Accuracy of LassoCV {:.3f}".format(lassoCV.score(X_test, y_test)))

        def imp_coef(model, n, columns=X_train.columns):
            array_to_df = pd.DataFrame(model.coef_)
            array_to_df.index = columns
            array_sorted = array_to_df.sort_values(0, ascending=False)
            imp_coef = pd.concat([array_sorted.head(int(n/2)), array_sorted.tail(int(n/2))])

            feature_importances = imp_coef.iloc[:,0].tolist()
            selected_features = imp_coef.index.tolist()
            return {"Lasso CV": [selected_features, feature_importances]}
        
        dict_ = imp_coef(lassoCV, n = n_features)
        return dict_
        
    def call_methods(self, X_train, y_train, X_test, y_test):
        method1 = self.gradient_boost_classifier(X_train, y_train, X_test, y_test, n_features = 300)
        method2 = self.rfe(X_train, y_train, X_test, y_test, kernel = "linear")
        method3 = self.elastic_net(X_train, y_train, X_test, y_test, alpha=0.01, l1_ratio=0.5, n_features = 300)
        method4 = self.boruta_tree(X_train, y_train, X_test, y_test)
        method5 = self.lasso_cv(X_train, y_train, X_test, y_test, n_features = 300)

        return { **method1, **method2, **method3, **method4, **method5}
        
# Instantiate Class Object
FS = FeatureSelection(1888)

# Class: Evaluation (evaluation)
Stores results (Counts of genes selected, importances, overlaps with cosmic) in a dataframe

In [ ]:
class Evaluation:
    """
    Included functions:
    - Add Cosmic to Dict: Add a dictionary with Cosmic cancer-related genes.
    - Results: Store results as csv file
    - Normalize Importances: Normalize of importances and add column with Total Importance
    - Final Results: Everything together in one df :)
    - Iterate Through Cancers: Iterate throuhg all cancer data
    """    
    def add_cosmic_to_dict(self, path, dict_list):
        """
        Add a dictionary with Cosmic cancer-related genes and corresponding Mutation Count
        """
        #path = "Data/Intogen_Data/Lung_Adenocarcinoma_LUAD_TCGA.tsv"
        census = pd.read_csv("Data/Reference_Data/Census_allWed May 15 09_46_55 2019.csv")
        census["GENE"] = census["Synonyms"].str.extract(pat = '(ENSG...........)')
        census = census[census["GENE"].notnull()]
        census.fillna("None", inplace = True)
        importances = census["Role in Cancer"].tolist()
        cosmic_genes = {"Cosmic":[census["GENE"].to_list(), importances]}
        dict_list = {**dict_list, **cosmic_genes}
        
        return dict_list
    
    def results(self, dict_list):
        """
        Store results in a csv
        Includes: Count for each method, feature importance, intogen counts
        """
        row_names = []
        column_names = []

        # Create dataframe with all viable genes from all method results
        for method, selected_features in dict_list.items():
            
            for feature in selected_features[0]:
                row_names.append(feature)

            row_names = list(set(row_names))
            column_names.append(method)

        results = pd.DataFrame(columns = column_names, index = row_names)
        results.fillna(0, inplace = True)
        
        # Add a one where the method selected the corresponding feature
        for method, selected_features in dict_list.items():
            for feature in selected_features[0]:
                results.at[feature, method] = 1

        # Create Column with total count
        results['Total Count'] = results[list(results.columns)].sum(axis=1)
        results.sort_values(by = "Total Count", ascending = False, inplace = True)
        
        # Add Importance Columns
        for method, selected_features in dict_list.items():
            additional = pd.DataFrame({"Importances: " + method:selected_features[1]}, index = selected_features[0])
            results = results.join(additional, how="outer")
            """if "key_0" in results.columns:
                results.drop(columns = "key_0", inplace = True)"""
        
        # Clean dataframe
        results.fillna(0, inplace = True)
        results = results.reset_index().drop_duplicates(subset='index', keep='first').set_index('index')
        results.rename(index=str, columns={"Importances: Cosmic": "Role in Cancer"}, inplace = True)
        
        return results
    
    def normalize_importance(self, result, threshold = 0):
        """
        This function returns the results table with normalized importances and an extra column - Total importance
        It is then sorted by total importance
        """
        #Normalize Importances
        imp = ['Importances: Gradient Boost Classifier',
               'Importances: Recursive Feature Elimination',
               'Importances: Elastic Net', 
               'Importances: Boruta Tree',
               'Importances: Lasso CV']
        
        scaler1 = MinMaxScaler() 
        scaler2 = StandardScaler(copy=True, with_mean=True, with_std=True)
        
        result[imp] = result[imp].abs()
        scaled_values = scaler1.fit_transform(result[imp]) 
        scaled_values = scaler2.fit_transform(scaled_values)
        
        result[imp] = scaled_values
        
        result['Importances: Recursive Feature Elimination'] = result['Importances: Recursive Feature Elimination']/4
        
        result["Importance Score"] = result[imp].sum(axis=1)
        result["Importance Score"] = scaler1.fit_transform(result[["Importance Score"]])
        result = result[result["Total Count"] > threshold].sort_values(by = "Importance Score", ascending = False)
        
        return result
    
    def final_results(self, path, path_intogen, nrows = 200, usecols = [x for x in range(100)], threshold = 3):
        """
        Puts everything together
        """
        X_train, y_train, X_test, y_test = dataprep.bulbasaur(path, threshold, nrows = nrows, usecols = usecols)
        dict_list = FS.call_methods(X_train, y_train, X_test, y_test)
        dict_list = evaluation.add_cosmic_to_dict(path_intogen, dict_list)
        df = evaluation.results(dict_list)
        df = evaluation.normalize_importance(df)
        return df

    def iterate_trough_cancers(self, path_list, path_intogen_list, nrows, usecols, threshold = 2.5):
        """
        Iterate through all cancer data
        """
        for path, path_intogen in zip(path_list, path_intogen_list):
            start = time.time()
            name = re.sub("^.+\/Chunk_", "2.0_", path)
            filepath = 'Output/Results/Result_{}'.format(name)

            print('Evaluating {}'.format(path))      

            results = evaluation.final_results(path, path_intogen, nrows = nrows, usecols = usecols, threshold = threshold)  
            results.to_csv(filepath)

            print('Finished in {:.1f} min\n'.format((time.time() - start) / 60))

evaluation = Evaluation()

# Apply and store results

In [ ]:
path_intogen_list = ["Data/Reference_Data/Census_allWed May 15 09_46_55 2019.csv"]#*7
path_list = ["Output/Chunk_Breast.csv",
            "Output/Chunk_LungAdenocarcinoma_Lung.csv",
            "Output/Chunk_LungSquamousCellCarcinoma_Lung.csv",
            "Output/Chunk_Skin.csv",
            "Output/Chunk_Thyroid_ThyroidGland.csv",
            "Output/Chunk_LungA1_vs_LungS1.csv",
            "Output/Chunk_AllCancers_0vs1.csv",
            "Output/Chunk_Colon.csv"]

In [ ]:
evaluation.iterate_trough_cancers(path_list, path_intogen_list, nrows = None, usecols = None, threshold = 1)